In [5]:
%pip install --quiet "sliderule @ git+https://github.com/SlideRuleEarth/sliderule#subdirectory=clients/python"

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install --quiet rtree


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --quiet hdbscan

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --quiet erddapy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from erddapy import ERDDAP
from sliderule import sliderule, icesat2, earthdata
import icepyx as ipx
from datetime import datetime, timedelta,timezone

import time
import os
import re
import glob
import pandas as pd
import numpy as np
import logging
import matplotlib.pyplot as plt

# from config import get_args

    
from kd_utils.data_processing import load_data, \
                                    extract_file_params, \
                                    Extract_sea_photons, \
                                    filter_photon_dataset_by_hull_area
                                    
    
from kd_utils.sea_photons_analysis import process_sea_photon_binning,\
                            get_sea_surface_height
                            
                            
from kd_utils.bathy_processing import process_subsurface_photon_filtering

from kd_utils.visualization import plot_photon_height, \
                                   plot_kd_photons, \
                                   plot_filtered_seafloor_photons
                                   
from kd_utils.Kd_analysis import process_kd_calculation

from kd_utils.interpolation import geoid_correction, \
                                    refraction_correction, \
                                    interpolate_labels, \
                                    apply_interpolation


# Configure logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
sliderule.init(verbose=False)

True

In [3]:
def fname2datetime(fname):
    y = int(fname[6:10])
    m = int(fname[10:12])
    d = int(fname[12:14])
    H = int(fname[14:16])
    M = int(fname[16:18])
    S = int(fname[18:20])

    t = datetime(y,m,d,H,M,S, tzinfo=timezone.utc)
    return t

def buoy_bound_box(lat,lon,buffer_km):
    # define a buffer distance around the buoy to search for icesat-2 data
    lat_buff = buffer_km/111 # convert buffer distance to frac of 1 deg lat
    lon_buff = buffer_km/(111*np.cos(lat*np.pi/180)) # convert buffer distance to frac of 1 deg lon
    # define bounding box around the buoy (WSEN)
    # example: bbox = [-108.3, 39.2, -107.8, 38.8]
    # bbox = [lon-lon_buff,lat+lat_buff,lon+lon_buff,lat-lat_buff]
    # region = sliderule.toregion(bbox)
    minx = lon - lon_buff
    miny = lat - lat_buff
    maxx = lon + lon_buff
    maxy = lat + lat_buff

    # polygon vertices: Given as longitude, latitude coordinate pairs of decimal degrees with the last entry a repeat of the first.
    poly_ipx = [(minx, miny), (maxx, miny), (maxx, maxy), (minx, maxy), (minx, miny)]
    poly_cmr = [{'lon': minx, 'lat': miny},
            {'lon': maxx, 'lat': miny},
            {'lon': maxx, 'lat': maxy},
            {'lon': minx, 'lat': maxy},
            {'lon': minx, 'lat': miny}] # Closing the loop by repeating the first point

    return poly_cmr, poly_ipx

def main():
    try:
        
        # Load IS2 and extract sea photons
        IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams = load_data(ATL03_h5_file_path)  
        
        ## For debugging
        #IS2_atl03_beams=['gt1r']
        
        # Check beam types and ensure we have what we want (e.g., strong beams)
        target_beam_type=['gt1l', 'gt2l', 'gt3l']        
        IS2_atl03_beams = [beam for beam in IS2_atl03_beams if beam in target_beam_type]
        if not IS2_atl03_beams:
            print('No Strong Beam')
            # sys.exit()  # Terminate the program
        else:
            print(f'Strong Beams Found: {IS2_atl03_beams}')
        
            # Extract sea photon by applying the land mask
            sea_photon_dataset = Extract_sea_photons(IS2_atl03_mds, IS2_atl03_beams, shoreline_data_path)
        
        
            # Main processing function to apply binning beam-by-beam 
            # to bin the photons data by vertical_res 25 cm and horizontal resolution 500m
            binned_dataset_sea_surface = process_sea_photon_binning(sea_photon_dataset, horizontal_res=horizontal_res, vertical_res=vertical_res)
                    
        
            # filter to get subsurface photons
            sea_surface_height, sea_surface_label, filtered_seafloor_subsurface_photon_dataset = \
                process_subsurface_photon_filtering(binned_dataset_sea_surface, subsurface_thresh, GEBCO_file_path_lists)
        
            
            # Visualization of the filtered seafloor photons
            # Uncomment the following lines to visualize if needed
            # output_path = OutputPath+'/IS2_subsurface_RM_Seafloor.jpg'
            # plot_filtered_seafloor_photons(filtered_seafloor_subsurface_dataset=filtered_seafloor_subsurface_photon_dataset, 
            #                                 sea_photon_dataset=sea_photon_dataset, 
            #                                 sea_surface_height=sea_surface_height, 
            #                                 output_path=output_path)
        
            ########################
            # Todo: move the filter to the begining
            # Filter shallow photons that may have noises from wave, afterpulse, or others
            # by setting Ignore_Subsurface_Height_Thres (i.e., -6)
            # Masked out the deep water photons by applying (filtered_seafloor_subsurface_photon_dataset < Ignore_Subsurface_Height_Thres)
            # Filter out points below the seafloor
            Final_filtered_subsurface_photon_dataset = filtered_seafloor_subsurface_photon_dataset[
                filtered_seafloor_subsurface_photon_dataset['photon_height'] < Ignore_Subsurface_Height_Thres
            ]
            
            # Final_filtered_subsurface_photon_dataset = filtered_seafloor_subsurface_photon_dataset
                   
            #Filter dataframe based on beam list
            Final_filtered_subsurface_photon_dataset = Final_filtered_subsurface_photon_dataset[Final_filtered_subsurface_photon_dataset['beam_id'].isin(target_beam_type)]
            
            ###################################
            ### Todo: move the filter to the begining
            # Steps to standardize KD calculation by filtering out 
            # segments lacking sufficient subsurface photons.
            # Approach: Use the ConvexHull of photons within 
            # each rectanglar area formed by each horizontal bin (here, it is lat_bins) and 
            # based on dataframe Final_filtered_subsurface_photon_dataset along the distance of the track
        
            # Apply filtering based on hull area if desired
            Final_filtered_subsurface_photon_dataset, convex_hull_areas, convex_hulls = filter_photon_dataset_by_hull_area(
                Final_filtered_subsurface_photon_dataset, hull_area_threshold=1500
            )
            
            # Visualization
            # Uncomment the following line to visualize if needed
            
            # plot_target_beam=['gt1l']
            # plot_convex_hulls(Final_filtered_subsurface_photon_dataset, plot_target_beam, convex_hulls, convex_hull_areas)
            
            ########################################### 
            # calculate kd and save it to table
            SubsurfacePhotonDFAddedKd = process_kd_calculation(Final_filtered_subsurface_photon_dataset)
        
        
            ## Todo move this part into a function so that it will not looks too busy here
            # Extract the timestamp using regex and use it to set the output file name
            match = re.search(r"_(\d{14})_", ATL03_h5_file_path)
            timestamp = match.group(1) if match else "unknown"
            
            output_file_path = os.path.join(OutputPath, f"{timestamp}_AddedKdDataset_strongBeams_Further.csv")
            SubsurfacePhotonDFAddedKd.to_csv(output_file_path, index=False)
        
            if 'relative_AT_dist' in filtered_seafloor_subsurface_photon_dataset.columns:
                unique_photon_dataset = filtered_seafloor_subsurface_photon_dataset[['relative_AT_dist', 'lat_bins', 'photon_height']].drop_duplicates()
                unique_photon_dataset['relative_AT_dist_center'] = unique_photon_dataset\
                                        .groupby('lat_bins', observed=False)['relative_AT_dist'].transform('mean')
                def find_closest(group):
                    if not group.empty:
                        center = group['relative_AT_dist_center'].iloc[0]
                        group['dist_to_center'] = abs(group['relative_AT_dist'] - center)
                        closest_index = group['dist_to_center'].idxmin()
                        return group.loc[[closest_index]]
                    else:
                        return pd.DataFrame(columns=group.columns)
                
                # Filter out empty or all-NA entries
                unique_photon_dataset_filtered = unique_photon_dataset.dropna(how='all', axis=1)
        
                # Group by 'lat_bins' and apply the find_closest function
                closest_to_center = unique_photon_dataset_filtered\
                                        .groupby('lat_bins', observed=False)\
                                        .apply(find_closest, include_groups=True)
                                        # .reset_index()
                                        
                # Remove the index without resetting it if 'lat_bins' already exists
                closest_to_center.index = closest_to_center.index.droplevel(0)
            
                # Ensure relevant entries are excluded before concatenation
                closest_to_center = closest_to_center.dropna(axis=1, how='all')
            
                Kd_DF_MergedDistance = closest_to_center.merge(SubsurfacePhotonDFAddedKd, on='lat_bins', how='left')
                Kd_DF_MergedDistance = Kd_DF_MergedDistance.drop(columns=['relative_AT_dist_center', 'dist_to_center'])
                
                
                # plot_kd_photons(OutputPath, timestamp, plot_target_beam, filtered_seafloor_subsurface_photon_dataset, Kd_DF_MergedDistance)
        
            logging.info("SUCCESS!")
    
    except Exception as e:
            logging.error(f"An error occurred: {e}")

# if __name__ == '__main__':
    
#     # Get arguments function
#     args = get_args()    
   
#     ATL03_h5_file_path = os.path.join(args.workspace_path, args.atl03_path, args.atl03_file)
    
#     shoreline_data_path = os.path.join(args.workspace_path, args.other_data_path, args.shoreline_data)
    
#     GEBCO_full_path = os.path.join(args.workspace_path, args.other_data_path, args.gebco_path)
    
#     OutputPath = os.path.join(args.workspace_path, args.output_path)
    
#     #Get the gebco list
#     #gebco_2024_n0.0_s-90.0_w0.0_e90.0.tif
#     GEBCO_file_pattern = os.path.join(GEBCO_full_path, "gebco_*.tif")

#     #Initialize with an empty list
#     GEBCO_file_path_lists = []
#     for GEBCO_file_path_name in glob.glob(GEBCO_file_pattern):
#         #put the find one into the list
#         GEBCO_file_path_lists.append(GEBCO_file_path_name)

    
#     # Get all other necessary parameters
#     horizontal_res = args.horizontal_res
    
#     vertical_res = args.vertical_res
    
#     subsurface_thresh = args.subsurface_thresh
    
#     Ignore_Subsurface_Height_Thres = args.ignore_subsurface_height_thres
    
#     # When all paramters ready, call the main function
#     main()


In [4]:
# load a list of all the relevant ERDDAPs and their urls
FF = pd.read_pickle("results/labeled_relevant_stations.pkl")
FF = FF[(FF.buoy == True) & ((FF.phot==True) | (FF.photo==True) | (FF.radiation==True))]
FF.reset_index(drop=True, inplace=True)
# # remove stations with PAR data in air or just no good data
FF = FF[27:42]
# FF = FF.drop(index=[34,35,36,37,39,40])
FF.reset_index(drop=True, inplace=True)

FF["geospatial_lat_min"] = pd.to_numeric(FF["geospatial_lat_min"])
FF["geospatial_lon_min"] = pd.to_numeric(FF["geospatial_lon_min"])
FF["geospatial_lat_max"] = pd.to_numeric(FF["geospatial_lat_max"])
FF["geospatial_lon_max"] = pd.to_numeric(FF["geospatial_lon_max"])
FF["photon_data"] = False

FF


,sites,url,time_coverage_start,time_coverage_end,geospatial_lat_min,geospatial_lat_max,geospatial_lon_min,geospatial_lon_max,geospatial_lat_units,geospatial_lon_units,...,glider,ship,turb,ssc,phot,chl,turbid,photo,radiation,photon_data
0,ooi-ce01issp-sp001-10-paradj000,https://erddap.dataexplorer.oceanobservatories...,2014-04-17T22:28:00Z,2023-09-22T08:00:00Z,44.660120,44.660120,-124.098120,-124.098120,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False
1,ooi-ce04osps-sf01b-3c-parada102,https://erddap.dataexplorer.oceanobservatories...,2015-08-03T15:10:00Z,2024-04-09T22:00:00Z,44.369353,44.369353,-124.954108,-124.954108,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False
2,ooi-ce02shsp-sp001-09-paradj000,https://erddap.dataexplorer.oceanobservatories...,2015-03-18T19:36:00Z,2023-09-23T08:08:00Z,44.635550,44.635550,-124.301470,-124.301470,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False
3,ooi-ce06issp-sp001-10-paradj000,https://erddap.dataexplorer.oceanobservatories...,2015-04-10T18:28:00Z,2023-09-23T08:02:00Z,47.135530,47.135530,-124.270170,-124.270170,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False
4,ooi-ce09ospm-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2014-04-19T01:02:00Z,2024-04-19T13:09:00Z,46.851630,46.851630,-124.974420,-124.974420,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False
5,ooi-ce07shsp-sp001-09-paradj000,https://erddap.dataexplorer.oceanobservatories...,2015-04-09T23:25:00Z,2022-04-26T20:04:00Z,46.983580,46.983580,-124.552130,-124.552130,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False
6,ooi-cp02pmci-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2014-04-17T01:02:00Z,2022-11-18T18:36:00Z,40.226633,40.226633,-70.889067,-70.889067,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False
7,ooi-cp02pmco-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2014-04-13T18:02:00Z,2022-11-18T15:30:00Z,40.096817,40.096817,-70.879150,-70.879150,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False
8,ooi-cp01cnpm-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2017-11-06T19:02:00Z,2022-04-24T18:27:00Z,40.133907,40.133907,-70.770433,-70.770433,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False
9,ooi-cp03ispm-wfp01-05-paradk000,https://erddap.dataexplorer.oceanobservatories...,2017-11-12T05:02:00Z,2022-04-25T18:24:00Z,40.362005,40.362005,-70.878502,-70.878502,degrees_north,degrees_east,...,False,False,False,False,True,False,False,True,True,False


In [ ]:
# these values can be adjusted to broaden/narrow the fit btwn icesat-2 and the ground truth
search_hrs = 6
search_km = 4
for jj in range(len(FF)):
    print('processing '+str(jj) +'/'+str(len(FF)))# give a printout every 100 for my sanity

    # define a search region around the buoy 
    lat = FF['geospatial_lat_min'][jj]
    lon = FF['geospatial_lon_min'][jj]
    poly_cmr,poly_ipx = buoy_bound_box(lat,lon,search_km)

    # search CMR for ATL03 granules in the bounding box
    grns = earthdata.cmr(short_name="ATL03",
                         polygon=poly_cmr,
                         version='006')
    # save the times for each granule as a datetime object
    icesat_times = [fname2datetime(fname) for fname in grns]

    # now check if buoy data exists for these granules,
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv")
    e.dataset_id = FF['sites'][jj]

    for t in icesat_times:
        # add a time buffer to search for relevant buoy data for each granule,
        t_start = (t-timedelta(hours=search_hrs)).strftime("%Y-%m-%dT%H:%M:%SZ")
        t_end = (t+timedelta(hours=search_hrs)).strftime("%Y-%m-%dT%H:%M:%SZ")
        e.constraints = {"time>=": t_start,
                        "time<=": t_end}
        # try to download the associated buoy data
        try:
            buoy = e.to_pandas()     
        except:
            continue
        
        # if buoy data exists, download the ATL03 photons in the bounding box at this time
        short_name = 'ATL03'
        t_start = (t-timedelta(hours=search_hrs))
        t_end = (t+timedelta(hours=search_hrs))
        date_range = [t_start,t_end]
        region = ipx.Query(short_name, poly_ipx, date_range)
        try:
            region.download_granules('data/')
        except:
            continue
        # buoy.to_csv('data/data_'+str(e.dataset_id)+'_'+str(t_start)+'.csv')


processing 0/15
Total number of data order requests is  1  for  1  granules.
Data request  1  of  1  is submitting to NSIDC


Enter your Earthdata Login username:  gloverha
Enter your Earthdata password:  ········


order ID:  5000005876110
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete
Beginning download of zipped output...
Data request 5000005876110 of  1  order(s) is downloaded.
Download complete
Total number of data order requests is  1  for  1  granules.
Data request  1  of  1  is submitting to NSIDC
order ID:  5000005876111
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete
Beginning download of zipped output...
Data request 5000005876111 of  1  order(s) is downloaded.
Download complete
Total number of 

In [ ]:
h5names = glob.glob('data/processed_*.h5')


for jj in range(len(h5names)):
    # code from chao (in Main.ipynb)
    atl03_file_inloop = h5names[jj]
    ATL03_h5_file_path = os.path.join(workspace_path, atl03_file_inloop)
    # print(ATL03_h5_file_path)
    shoreline_data_path = os.path.join(workspace_path, other_data_path, shoreline_data)
    # print(shoreline_data_path)
    GEBCO_full_path = os.path.join(workspace_path, other_data_path, gebco_path)
    # print(GEBCO_full_path)
    OutputPath = os.path.join(workspace_path, output_path)
    # print(OutputPath)
    
    
    #Get the gebco list
    GEBCO_file_pattern = os.path.join(GEBCO_full_path, "gebco_*.tif")
    
    #Initialize with an empty list
    GEBCO_file_path_lists = []
    for GEBCO_file_path_name in glob.glob(GEBCO_file_pattern):
        #put the find one into the list
        GEBCO_file_path_lists.append(GEBCO_file_path_name)
    
    # print(GEBCO_file_path_lists)
    
    # Get all other necessary parameters
    horizontal_res = horizontal_res
    vertical_res = vertical_res
    subsurface_thresh = subsurface_thresh
    Ignore_Subsurface_Height_Thres = ignore_subsurface_height_thres
    
    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    
    # When all paramters ready, call the main function
    main()
    # delete the hefty h5 file once done with processing - only store calculated kd values
    os.remove(ATL03_h5_file_path)